## Support Vector Machines for Classification
### Hard Margin (SVM without regularization)

### Soft Margin (SVM with regularization)

### Simplified Sequential Minimal Optimization (SMO)


In [1]:
%matplotlib inline
import numpy as np 
import sklearn.preprocessing
import sklearn.datasets
import pandas as pd
import sklearn.model_selection
import numpy.random
import math
import sklearn.metrics

In [2]:
X, y = sklearn.datasets.load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, random_state=42)
standard = sklearn.preprocessing.StandardScaler()
X_train = standard.fit_transform(X_train)
training_data = pd.DataFrame(np.c_[X_train, y_train])#All of the features are continuous, so, no need to use one-hot encoder and we can directly standard normalize the features of the data set
y_train = np.array(list(map(lambda y: 1 if y == 1 else -1, y_train)))# y must be either 1 or -1 

X_test = standard.transform(X_test)
y_test = np.array(list(map(lambda y: 1 if y == 1 else -1, y_test)))# y must be either 1 or -1 
test_data = pd.DataFrame(np.c_[X_test, y_test])
print(training_data.shape)
print(test_data.shape)
#training_data.iloc[10]

(426, 31)
(143, 31)


In [3]:
class SVM2CLASS(object):

    def __init__(self, X_train, y_train, C = 10, tol = 0.001, max_passes = 5, passes = 0):
        
        self.X_train = X_train
        self.y_train = y_train
        self.C = C
        self.tol = tol
        self.max_passes = max_passes
        self.passes = passes
        self.b = 0
        self.alphas = np.zeros((self.X_train.shape[0], 1))#need to be of size nx1
        self.coefficients = ()
        self.kernel_type ={"poly": self.polynomial_kernel, "gauss": self.gaussian_kernel}
        self.kernel_parameters = []
        self.kernel_choice = ""

    def polynomial_kernel(self, x_i, x_j, a):
        return np.power(np.dot(x_i.T, x_j) + a[0], a[1])

    def gaussian_kernel(self, x, x_star, sigma):
        return np.exp(np.divide(-1*(np.linalg.norm(x-x_star)**2), 2*sigma**2))
    
    def SMO(self, kernel_choice, parameters):

        choices = np.arange(0, self.y_train.shape[0])
        count = 0
        self.kernel_choice = kernel_choice
        #Better to construct the Kernel matrix from the scratch for efficiency, but this method will prevent the simplified SMO from working on large datasets, like, the mnist dataset 
        if kernel_choice == "poly":
            exponent = parameters[0]
            intercept = parameters[1]
            self.kernel_parameters = [intercept, exponent] 
            
            K = np.zeros((self.X_train.shape[0], self.X_train.shape[0]))
            for i in range(0, self.X_train.shape[0]):
                for j in range(0, self.X_train.shape[0]):
                    K[i, j] = self.kernel_type[self.kernel_choice](self.X_train[i, :], self.X_train[j, :], self.kernel_parameters)
            assert(np.any(np.linalg.eig(K)[0] == 0)  == False)#Test for PSD

        elif kernel_choice == "gauss":
            self.kernel_parameters = 2

            K = np.zeros((self.X_train.shape[0], self.X_train.shape[0]))
            for i in range(0, self.X_train.shape[0]):
                for j in range(0, self.X_train.shape[0]):
                    K[i, j] =  self.kernel_type[self.kernel_choice](self.X_train[i, :], self.X_train[j, :], self.kernel_parameters)
            assert(np.linalg.det(K) != 0)#Test for PSD
        
        else:
            print("Wrong entry")
            return -1

        while(self.passes <= self.max_passes):
        #begin while
            num_changed_alphas = 0
            for i in range(0, self.X_train.shape[0]):
            #begin for
                #compute kernel function for every iteration from scratch
                #f_x_i = np.sum( list(map(lambda x, alpha, y: alpha * y * self.kernel_type[self.kernel_choice](x, self.X_train[i, :], self.kernel_parameters) , self.X_train, self.alphas, self.y_train) ) )#instead of calculating the Kernel matrix from the start, we will calculate the inner product with each iteration in order to mitigate the problem of having a large kernel matrix that will raise an exception
                #print(f_x_i)
                f_x_i = np.sum(self.alphas.reshape(-1, 1) * (self.y_train.reshape(-1, 1) *  K[:, i].reshape(-1, 1)).reshape(-1, 1)) 
                E_i = f_x_i + self.b - self.y_train[i]
                #print(f_x_i)
                #Check if we satisfy the condition for the dual problem
                if (((self.y_train[i] * E_i) < -self.tol) and (self.alphas[i] < self.C)) or (((self.y_train[i] * E_i) > self.tol) and (self.alphas[i] > 0)):
                #begin if
                    j = np.random.choice( list(filter(lambda v: v == v, list(map(lambda c: c if c != i else np.nan, choices)))) ) 
                    #only nan will generate False at its equlaity, and the filter object will end up filtering out these wrong values
                    assert( i != j)
                    #f_x_j = np.sum( list(map(lambda x, alpha, y: alpha * y * self.kernel_type[self.kernel_choice](x, self.X_train[j, :], self.kernel_parameters) , self.X_train, self.alphas, self.y_train) ) )
                    #print(f_x_j)
                    f_x_j = np.sum(self.alphas.reshape(-1, 1) * (self.y_train.reshape(-1, 1) * K[:, j].reshape(-1, 1)).reshape(-1, 1)) 
                    #print((alphas.reshape(-1, 1) * (y_train.reshape(-1, 1) * K[i, :].reshape(-1, 1)).reshape(-1, 1)).shape)
                    #print(f_x_j)
                    E_j = f_x_j + self.b - self.y_train[j]

                    alpha_i_old = self.alphas[i].copy()#Needs to copy the value because otherwise they would be pointing to the same address
                    alpha_j_old = self.alphas[j].copy()

                    #Computing L and H
                    if(self.y_train[i] != self.y_train[j]):
                        L = max(0, self.alphas[j] - self.alphas[i])
                        H = min(self.C, self.C + self.alphas[j] - self.alphas[i])
                    else:
                        L = max(0, self.alphas[j] + self.alphas[i] - self.C)
                        H = min(self.C, self.alphas[j] + self.alphas[i])
                    
                    #Checking if L=H which indicate that the alpha would certainly wouldn't change 
                    if L == H:
                        continue

                    #eta = 2 * self.kernel_type[self.kernel_choice](self.X_train[i, :], self.X_train[j, :], self.kernel_parameters) - self.kernel_type[self.kernel_choice](self.X_train[i, :], self.X_train[i, :], self.kernel_parameters) - self.kernel_type[self.kernel_choice](self.X_train[j, :], self.X_train[j, :], self.kernel_parameters)
                    eta = 2 * K[i, j] - K[i, i] - K[j, j] 

                    #eta = 0 if the similarity between x_i and x_j is as the combination of the similarity of x_i with itself and same goes for x_j, will cause an exception to happend and this indicate we are dealing with the same x.
                    #eta > 0 if if the similarity between x_i and x_j is higher than the combination of the similarity of x_i with itself and same goes for x_j, so, this update step would have little effect on the converging to the optimal minimum and may leads to diverging the algorithm patht to a worse path
                    #eta < 0 there are small simialrity between x_i and x_j, so, this would help in discoverign the interaction of those observations in the feature space

                    if eta >= 0:
                        #print("The two vectors are too similar")
                        continue

                    alpha_j_clip = alpha_j_old - (1/eta) * self.y_train[j] * (E_i - E_j)

                    if alpha_j_clip > H:
                        self.alphas[j] = H
                    elif alpha_j_clip < L:
                        self.alphas[j] = L
                    else:
                        self.alphas[j]  = alpha_j_clip
                    
                    #print(alphas[j], alpha_j_old)
                    #Check if it is worth to update alpha_i
                    if(abs(self.alphas[j] - alpha_j_old) < 1e-3):
                        #print("No noticeable changes happened to alpha")
                        continue
                    self.alphas[i] = alpha_i_old + self.y_train[i] * self.y_train[j] * (alpha_j_old - self.alphas[j])#The signs changed from  the negative sign for updating alpha_i

                    ##KKT constrains convergence test
                    #b1 = self.b - E_i - self.y_train[i]*(self.alphas[i] - alpha_i_old) * self.kernel_type[self.kernel_choice](self.X_train[i, :], self.X_train[i, :], self.kernel_parameters) - self.y_train[j]*(self.alphas[j] - alpha_j_old) * self.kernel_type[self.kernel_choice](self.X_train[i, :], self.X_train[j, :], self.kernel_parameters)
                    b1 = self.b - E_i - self.y_train[i] * (self.alphas[i] - alpha_i_old) * K[i, i] - self.y_train[j] * (self.alphas[j] - alpha_j_old) * K[i, j]

                    #b2 = self.b - E_j - self.y_train[i]*(self.alphas[i] - alpha_i_old) * self.kernel_type[self.kernel_choice](self.X_train[i, :], self.X_train[j, :], self.kernel_parameters) - self.y_train[j]*(self.alphas[j] - alpha_j_old) * self.kernel_type[self.kernel_choice](self.X_train[j, :], self.X_train[j, :], self.kernel_parameters)
                    b2 = self.b - E_j - self.y_train[i] * (self.alphas[i] - alpha_i_old) * K[i, j] - self.y_train[j] * (self.alphas[j] - alpha_j_old) * K[j, j]

                    if (self.alphas[j] > 0) and (self.alphas[j] < self.C):
                        self.b = b2
                    elif (self.alphas[i] > 0) and (self.alphas[i] < self.C):
                        self.b = b1
                    else:
                        self.b = (b1 + b2)/2
                    
                    num_changed_alphas  =  num_changed_alphas + 1
                #end if
            #end for
            print(f"count:{count}, passes:{self.passes}, max_passes:{self.max_passes}, b:{self.b}")
            count+=1
            if(num_changed_alphas == 0):
                self.passes = self.passes + 1
            else:
                self.passes = 0
        #end while
        #Only store in the memory the support vectors
        support_indeces = np.argwhere(self.alphas != 0)[:, 0]
        support_vectors = self.X_train[support_indeces, :]
        support_alphas = self.alphas[support_indeces]
        support_target = self.y_train[support_indeces]
        self.importantParameters = (support_vectors, support_alphas, support_target)

        return self.importantParameters

    def prediction_dataset(self, X):
        pred = list(map(lambda x: self.prediction(x), X))
        return pred

    def prediction(self, x):
        t1 = np.sum( list(map(lambda x1, alpha, y: y * alpha * self.kernel_type[self.kernel_choice](x, x1, self.kernel_parameters), self.importantParameters[0], self.importantParameters[1], self.importantParameters[2])) )
        pred = t1 + self.b
        #print(pred)
        if pred >=0:
            return 1
    
        return -1

In [4]:
svm_model =SVM2CLASS(X_train, y_train, C = 10, tol = 0.001, max_passes = 5, passes = 0)
support_vectors, support_alphas, support_target = svm_model.SMO("poly", [1, 1])
pred = svm_model.prediction_dataset(X_train)
print("Performance on the training set")
print(sklearn.metrics.confusion_matrix(y_train, pred))

, max_passes:5, b:[-0.15087567]
count:3371, passes:2, max_passes:5, b:[-0.24400035]
count:3372, passes:0, max_passes:5, b:[-0.24400035]
count:3373, passes:1, max_passes:5, b:[-0.19336467]
count:3374, passes:0, max_passes:5, b:[-0.19336467]
count:3375, passes:1, max_passes:5, b:[-0.33252898]
count:3376, passes:0, max_passes:5, b:[-0.14952051]
count:3377, passes:0, max_passes:5, b:[-0.25345214]
count:3378, passes:0, max_passes:5, b:[-0.25345214]
count:3379, passes:1, max_passes:5, b:[-0.06350016]
count:3380, passes:0, max_passes:5, b:[-0.11353621]
count:3381, passes:0, max_passes:5, b:[-0.28020908]
count:3382, passes:0, max_passes:5, b:[-0.28020908]
count:3383, passes:1, max_passes:5, b:[-0.30796845]
count:3384, passes:0, max_passes:5, b:[-0.3291488]
count:3385, passes:0, max_passes:5, b:[-0.3291488]
count:3386, passes:1, max_passes:5, b:[-0.25832276]
count:3387, passes:0, max_passes:5, b:[-0.15244854]
count:3388, passes:0, max_passes:5, b:[-0.15244854]
count:3389, passes:1, max_passes:5

In [5]:
pred = svm_model.prediction_dataset(X_train)
print("Performance on the training set")
print(sklearn.metrics.confusion_matrix(y_train, pred))

Performance on the training set
[[156   2]
 [  0 268]]


In [6]:
pred = svm_model.prediction_dataset(X_test)
print("Performance on the test set")
print(sklearn.metrics.confusion_matrix(y_test, pred))

Performance on the test set
[[52  2]
 [ 4 85]]


In [8]:
import sklearn.svm

def polynomial_kernel(x_i, x_j, a):
        return np.power(np.dot(x_i.T, x_j) + a[0], a[1])

K = np.zeros((X_train.shape[0], X_train.shape[0]))
for i in range(0, X_train.shape[0]):
    for j in range(0, X_train.shape[0]):
        K[i, j] = polynomial_kernel(X_train[i, :], X_train[j, :], [0, 1])
assert(np.any(np.linalg.eig(K)[0] == 0)  == False)#Test for PSD

model = sklearn.svm.SVC(kernel="precomputed", C=10)
model.fit(K, y_train)
print("Performance on the training set")
print(sklearn.metrics.confusion_matrix(y_train, model.predict(K)))
print(model.support_)
print(model.intercept_)
model.dual_coef_

Performance on the training set
[[156   2]
 [  1 267]]
[  5  33  91 105 126 164 168 303 325 335 365 374 384 412  38  43  53  65
 107 115 120 134 151 250 318 353 407 418]
[-0.06041561]


array([[ -1.90700976,  -3.37970192,  -2.10920559, -10.        ,
         -2.53562665, -10.        ,  -0.27063242,  -2.75357763,
        -10.        , -10.        ,  -3.43593007,  -3.32707653,
         -6.57192222,  -3.62688542,   5.54572154,   3.34438379,
          6.72527011,   5.01150459,  10.        ,   2.1042078 ,
          3.52560421,   0.27223051,   7.82312681,   0.38694323,
          0.21750621,   9.57067784,  10.        ,   5.39039159]])

### References 
* Chapter 1, chapter 6 and Chapter 7 from Bishop, C. (2006). Pattern Recognition and Machine Learning. Cambridge: Springer.
* Andrew Ng, Lec 6: (https://www.youtube.com/watch?v=qyyJKd-zXRE)
* Andrew Ng, Lec 7: (https://www.youtube.com/watch?v=s8B4A5ubw6c)
* Andrew Ng, Lec 8: (https://www.youtube.com/watch?v=bUv9bfMPMb4)
* Simplified Sequential Minimal Optimization: (https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&cad=rja&uact=8&ved=2ahUKEwiRlObmw5_qAhW7ShUIHSjJAbYQFjAAegQIAhAB&url=http%3A%2F%2Fcs229.stanford.edu%2Fmaterials%2Fsmo.pdf&usg=AOvVaw201bQxVZY0MmUn_gGAu5O8)
